In [1]:
import tensorflow as tf

In [2]:
print(tf.__version__)

2.17.0


In [3]:
import nltk
nltk.download("punkt_tab")
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# Import necessary libraries
import tensorflow as tf
import numpy as np
import random
import json
import pickle

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [5]:
with open("intents.json") as json_data:
    intents = json.load(json_data)

In [6]:
# Empty lists for appending the data after processing NLP
words = []
classes = []
documents = []
ignore = ["?"]

# Tokenize and process each pattern
for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent["tag"]))
        if intent["tag"] not in classes:
            classes.append(intent["tag"])

In [7]:
#Performing Stemming by using stemmer.stem() nd lower each word
#Running loop in words[] and ignoring punctuation marks present in ignore[]
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]

#Removing Duplicate
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

In [8]:
print(len(documents), "documents")
print(len(classes), "classes")
print(len(words), "unique stemmed words")


106 documents
37 classes
118 unique stemmed words


In [9]:
#Creating Training Data which will be furthur used for training
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0] #Storing list of tokenized words for the documents[] tp pattern_words

       #Again Stemming each word from pattern_words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]

      #Creating bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    #It will give output 1 for curent tag and 0 for all other tags
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])


In [10]:
random.shuffle(training) #Suffling the data or features

# Separate the data into train_x and train_y
train_x = []
train_y = []

for feature, label in training:
    train_x.append(feature)  # Bag of words (features)
    train_y.append(label)    # Output row (labels)

# Convert to numpy arrays
train_x = np.array(train_x)
train_y = np.array(train_y)


In [11]:

# Define the Keras model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [13]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)
print("Eager execution enabled:", tf.executing_eagerly())

Eager execution enabled: True


In [14]:
import tensorflow as tf
import numpy as np

# Ensure eager execution is enabled (this is default in TensorFlow 2.x)
tf.config.run_functions_eagerly(True)

# Define a simple neural network with Keras layers
input_dim = len(train_x[0])
output_dim = len(train_y[0])

# Creating the model
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(input_dim,)),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(output_dim, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Convert training data to numpy arrays
train_x = np.array(train_x)
train_y = np.array(train_y)

# Define the custom training step using tf.function
@tf.function
def train_step(x_batch, y_batch):
    with tf.GradientTape() as tape:
        # Forward pass
        predictions = model(x_batch, training=True)
        loss = tf.keras.losses.categorical_crossentropy(y_batch, predictions)

    # Compute gradients
    gradients = tape.gradient(loss, model.trainable_variables)

    # Apply gradients (updating the weights)
    model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss

# Training loop without using tf.data.Dataset
def custom_training(train_x, train_y, epochs, batch_size):
    num_samples = train_x.shape[0]
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        for step in range(0, num_samples, batch_size):
            x_batch = train_x[step:step + batch_size]
            y_batch = train_y[step:step + batch_size]
            loss = train_step(x_batch, y_batch)
            if step % 100 == 0:
                print(f"Step {step}: Loss = {tf.reduce_mean(loss)}")

# Train the model using the custom training loop
custom_training(train_x, train_y, epochs=100, batch_size=8)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/100
Step 0: Loss = 3.6318752765655518
Epoch 2/100
Step 0: Loss = 3.6033129692077637
Epoch 3/100
Step 0: Loss = 3.5906548500061035
Epoch 4/100
Step 0: Loss = 3.5799622535705566
Epoch 5/100
Step 0: Loss = 3.569514513015747
Epoch 6/100
Step 0: Loss = 3.5589680671691895
Epoch 7/100
Step 0: Loss = 3.546220064163208
Epoch 8/100
Step 0: Loss = 3.5304908752441406
Epoch 9/100
Step 0: Loss = 3.5119881629943848
Epoch 10/100
Step 0: Loss = 3.4933266639709473
Epoch 11/100
Step 0: Loss = 3.472608804702759
Epoch 12/100
Step 0: Loss = 3.4503376483917236
Epoch 13/100
Step 0: Loss = 3.4260785579681396
Epoch 14/100
Step 0: Loss = 3.397939682006836
Epoch 15/100
Step 0: Loss = 3.3645267486572266
Epoch 16/100
Step 0: Loss = 3.326995372772217
Epoch 17/100
Step 0: Loss = 3.2823026180267334
Epoch 18/100
Step 0: Loss = 3.233091115951538
Epoch 19/100
Step 0: Loss = 3.1797895431518555
Epoch 20/100
Step 0: Loss = 3.1241707801818848
Epoch 21/100
Step 0: Loss = 3.065528392791748
Epoch 22/100
Step 0: Loss = 3

In [15]:
tf.config.run_functions_eagerly(True)
import keras.saving
keras.saving.save_model(model, "C:/Users/Shruti Jain/OneDrive/Desktop/proj/proj.h5")
pickle.dump({"words": words, "classes": classes, "train_x": train_x, "train_y": train_y}, open("training_data", "wb"))

In [16]:
from tensorflow.keras.models import load_model
import pickle

model = load_model("C:/Users/Shruti Jain/OneDrive/Desktop/proj/proj.h5")

# Recompile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Loading training data
data = pickle.load(open("training_data", "rb"))
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']



In [17]:
# Function to clean up and tokenize user input
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words



In [18]:
# Bag of words function
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("Found in bag: %s" % w)
    return np.array(bag)


In [19]:
# Classify function to predict intent
ERROR_THRESHOLD = 0.25
def classify(sentence):
    bow_data = bow(sentence, words)
    results = model.predict(np.array([bow_data]))[0]
    results = [[i, r] for i, r in enumerate(results) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = [(classes[r[0]], r[1]) for r in results]
    return return_list


In [20]:
# Response function to generate response
context = {}
def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    if results:
        while results:
            for i in intents['intents']:
                if i['tag'] == results[0][0]:
                    if 'context_set' in i:
                        context[userID] = i['context_set']
                    if not 'context_filter' in i or (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        return random.choice(i['responses'])
            results.pop(0)

In [21]:
response("Hello")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


'Hello, thanks for visiting'

In [22]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [25]:
import gradio as gr
def chat_model(user_input, history):
    # Get bot response
    bot_response = response(user_input)
    # Add to conversation history
    history.append(f"You: {user_input}")
    history.append(f"Bot: {bot_response}")

    # Create a conversation string to display
    conversation = "\n".join(history)

    return conversation, history  # Return the conversation and updated history


# Define Gradio Interface
interface = gr.Interface(
    fn=chat_model,                # Function to call on input
    inputs=["text", "state"],     # Input type (text box for user input) and state for conversation history
    outputs=["text", "state"],    # Output type (conversation history and updated history state)
    title="Chat with the Model",  # Title of the chat UI
    description="Type a message to chat with the model!",  # Short description
    live=False                    # Disable live mode, trigger only on Enter key
)

# Launch the Gradio interface
interface.launch(share=True)  #

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a04e960f5fa5a25477.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
